## Setup

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('PySparkShell').getOrCreate()
sc = spark.sparkContext

In [2]:
import datetime
import operator
import time

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType, TimestampType, ShortType
from pyspark.sql.types import IntegerType, LongType, StructField, StructType

In [3]:
artists_schema = StructType([
    StructField('artistId', LongType(), False),
    StructField('artistName', StringType(), True),
])

chart_artist_mapping_schema = StructType([
    StructField('songId', LongType(), False),
    StructField('artistId', LongType(), True),
])

charts_schema = StructType([
    StructField('songId', LongType(), False),
    StructField('title', StringType(), True),
    StructField('position', ShortType(), True),
    StructField('date', DateType(), True),
    StructField('countryId', LongType(), True),
    StructField('chartName', StringType(), True),
    StructField('movement', StringType(), True),
    StructField('streams', LongType(), True),
])

regions_schema = StructType([
    StructField('countryId', LongType(), False),
    StructField('countryName', StringType(), True),
])

# # Write parquet files
# it = [
#     ('artists', artists_schema),
#     ('chart_artist_mapping', chart_artist_mapping_schema),
#     ('charts', charts_schema),
#     ('regions', regions_schema),
# ]

# for stem, schema in it:
#     print(stem)
    
#     df = spark.read.csv(
#         'files/{}.csv'.format(stem),
#         sep=',', header='false', schema=schema
#     )
#     df.write.save('files/{}.parquet'.format(stem))
    
#     spark.read.parquet('files/{}.parquet'.format(stem)).show(2)
#     print('=' * 40)

In [4]:
def parse_date(s):
    dt = datetime.datetime.strptime(s[:10], '%Y-%m-%d')
    return dt.date()


def tuple_to_string(t):
    return ','.join(str(x) for x in t)


def timedSql(query, path):
    tStart = time.perf_counter()
    spark.sql(query).coalesce(1).write.csv(
        path, header=True, sep=',', mode='overwrite'
    )
    tEnd = time.perf_counter()
    elapsed = tEnd - tStart
    print('{:.2f} seconds elapsed'.format(elapsed))
    return elapsed


def timedRdd(rdd, path):
    tStart = time.perf_counter()
    rdd.map(tuple_to_string).coalesce(1).saveAsTextFile(path)
    tEnd = time.perf_counter()
    elapsed = tEnd - tStart
    print('{:.2f} seconds elapsed'.format(elapsed))
    return elapsed

In [5]:
# Aliases for easier indexing
songId = 0
title = 1
position = 2
date = 3
countryId = 4
chartName = 5
movement = 6
streams = 7

In [6]:
artists = spark.read.parquet('files/artists.parquet')
artistsCsv = spark.read.csv(
    'files/artists.csv',
    sep=',', header='false', schema=artists_schema
)
artistsRdd = sc.textFile('files/artists.csv').map(lambda line: line.split(','))

chart_artist_mapping = spark.read.parquet('files/chart_artist_mapping.parquet')
chart_artist_mappingCsv = spark.read.csv(
    'files/chart_artist_mapping.csv',
    sep=',', header='false', schema=chart_artist_mapping_schema
)
chart_artist_mappingRdd = (
    sc.textFile('files/chart_artist_mapping.csv')
      .map(lambda line: line.split(','))
)

charts = spark.read.parquet('files/charts.parquet')
chartsCsv = spark.read.csv(
    'files/charts.csv',
    sep=',', header='false', schema=charts_schema
)
chartsRdd = sc.textFile('files/charts.csv').map(lambda line: line.split(','))

regions = spark.read.parquet('files/regions.parquet')
regionsCsv = spark.read.csv(
    'files/regions.csv',
    sep=',', header='false', schema=regions_schema
)
regionsRdd = sc.textFile('files/regions.csv').map(lambda line: line.split(','))


def useParquet():
    charts.createOrReplaceTempView('charts')
    regions.createOrReplaceTempView('regions')
    chart_artist_mapping.createOrReplaceTempView('chart_artist_mapping')
    artists.createOrReplaceTempView('artists')

    
def useCsv():
    chartsCsv.createOrReplaceTempView('charts')
    regionsCsv.createOrReplaceTempView('regions')
    chart_artist_mappingCsv.createOrReplaceTempView('chart_artist_mapping')
    artistsCsv.createOrReplaceTempView('artists')

In [ ]:
times = {'query{}'.format(i): {'parquet': None, 'csv': None, 'rdd': None}
         for i in range(1, 7)}

## Query 1

In [ ]:
query1 = '''
    SELECT sum(streams) FROM charts
    WHERE chartName="top200" AND title="Shape of You"
'''

In [ ]:
useParquet()
times['query1']['parquet'] = timedSql(query1, 'output/query1_parquet')

In [ ]:
useCsv()
times['query1']['csv'] = timedSql(query1, 'output/query1_csv')

In [ ]:
t0 = time.perf_counter()
res1 = (
    chartsRdd
     .filter(lambda x: x[chartName] == 'top200' and x[title] == 'Shape of You')
     .map(lambda x: int(x[streams]))
     .reduce(operator.add)
)
t1 = time.perf_counter()
times['query1']['rdd'] = t1 - t0
sc.parallelize([str(res1)]).coalesce(1).saveAsTextFile('output/query1_rdd')

## Query 2

In [19]:
query2 = '''
    SELECT chartName, title, avgCount
    FROM (
        SELECT chartName, title, avgCount,
            max(avgCount) OVER (PARTITION BY chartName) AS maxAvgCount
        FROM (
            SELECT chartName, first(title) title, count(*)/69 avgCount
            FROM charts
            WHERE position=1
            GROUP BY chartName, songId
        )
    )
    WHERE avgCount = maxAvgCount
'''

In [ ]:
useParquet()
times['query2']['parquet'] = timedSql(query2, 'output/query2_parquet')

In [ ]:
useCsv()
times['query2']['csv'] = timedSql(query2, 'output/query2_csv')

In [ ]:
rdd2 = (
    chartsRdd
     .filter(lambda x: x[position] == '1')
     .map(lambda x: ((x[chartName], x[title]), 1))
     .reduceByKey(operator.add)
     .map(lambda x: (x[0][0], (x[0][1], x[1]/69)))
     .reduceByKey(lambda x, y: x if x[1] > y[1] else y)
     .map(lambda x: (x[0], *x[1]))
)

times['query2']['rdd'] = timedRdd(rdd2, 'output/query2_rdd')

## Query 3

In [ ]:
query3 = '''
    SELECT year(date), month(date), sum(streamsDay) streams1, count(*)
    FROM (
        SELECT date, sum(streams) streamsDay
        FROM charts
        WHERE position = 1 AND chartName == "top200"
        GROUP BY date
    )
    GROUP BY year(date), month(date)
    ORDER BY year(date), month(date)
'''

In [ ]:
useParquet()
times['query3']['parquet'] = timedSql(query3, 'output/query3_parquet')

In [ ]:
useCsv()
times['query3']['csv'] = timedSql(query3, 'output/query3_csv')

In [ ]:
rdd3 = (
    chartsRdd
     .filter(lambda x: x[position] == '1' and x[chartName] == 'top200')
     .map(lambda x: (parse_date(x[date]), int(x[streams])))
     .reduceByKey(operator.add)
     .map(lambda x: ((x[0].year, x[0].month), (x[1], 1)))
     .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
     .sortByKey()
     .map(lambda x:(x[0][0], x[0][1], x[1][0] / x[1][1]))
)
times['query3']['rdd'] = timedRdd(rdd3, 'output/query3_rdd')

## Query 4

In [ ]:
query4 = '''
    SELECT countryName, songId songId, title, maxCount
    FROM (
        SELECT countryId, songId, title, cnt, 
            max(cnt) OVER (PARTITION BY countryId) AS maxCount 
        FROM (
            SELECT countryId, songId, first(title) title, count(*) cnt
            FROM charts
            WHERE chartName = "viral50"
            GROUP BY countryId, songId
        )
    ) a
    LEFT JOIN regions
    ON a.countryId = regions.countryId
    WHERE cnt = maxCount
    ORDER BY countryName, title
'''

In [ ]:
useParquet()
times['query4']['parquet'] = timedSql(query4, 'output/query4_parquet')

In [ ]:
useCsv()
times['query4']['csv'] = timedSql(query4, 'output/query4_csv')

In [ ]:
# countryId, (songId, title, count)
counts = (
    chartsRdd
     .filter(lambda x: x[chartName] == 'viral50')
     .map(lambda x: ((x[countryId], x[songId]), (x[title], 1)))
     .reduceByKey(lambda x, y: (x[0], x[1] + y[1]))
     .map(lambda x: (x[0][0], (x[0][1], *x[1])))
)

# countryId, maxCount
maxCounts = (
    counts
     .map(lambda x: (x[0], x[1][2]))
     .reduceByKey(max)
)

# Joining now because both tables are tiny and of equal index

# countryId, (countryName, maxCount)
maxCountsNamed = regionsRdd.join(maxCounts)

rdd4 = (
    counts
     .join(maxCountsNamed)
     # countryId, ((songId, title, count), (countryName, maxCount))
     .filter(lambda x: x[1][0][2] == x[1][1][1])  # count == maxCount
     .map(lambda x: (x[1][1][0], x[1][0][0], x[1][0][1], x[1][1][1]))
     # countryName, songId, songName, maxCount
     .sortBy(lambda x: (x[0], x[1]))
     # countryName, songId, songName, maxCount
)

times['query4']['rdd'] = timedRdd(rdd4, 'output/query4_rdd')

## Query 5

In [ ]:
query5 = '''
    SELECT yr, artistName, maxAvgStreams
    FROM (
        SELECT yr, artistId, avgStreams,
            max(avgStreams) OVER (PARTITION BY yr) AS maxAvgStreams 
        FROM (
            SELECT year(date) yr, artistId, sum(streams)/69 avgStreams
            FROM charts
            JOIN chart_artist_mapping
            ON charts.songId = chart_artist_mapping.songId
            WHERE chartName = "top200"
            GROUP BY year(date), artistId
        )
    ) a
    JOIN artists
    ON a.artistId = artists.artistId
    WHERE avgStreams = maxAvgStreams
    ORDER BY yr
'''

In [ ]:
useParquet()
times['query5']['parquet'] = timedSql(query5, 'output/query5_parquet')

In [ ]:
useCsv()
times['query5']['csv'] = timedSql(query5, 'output/query5_csv')

In [ ]:
avgStreams = (
    chartsRdd
     .filter(lambda x: x[chartName] == 'top200')
     .map(lambda x: (x[songId], (parse_date(x[date]).year, int(x[streams]))))
     .join(chart_artist_mappingRdd)
     # songId, ((year, streams), artistId)
     .map(lambda x: ((x[1][0][0], x[1][1]), x[1][0][1]/69))
     # (year, artistId), streams
     .reduceByKey(operator.add)
     # (year, artistId), avgStreams
)

maxAvgStreams = (
    avgStreams
     .map(lambda x: (x[0][0], x[1]))
     .reduceByKey(max)
     # year, maxAvgStreams
)

rdd5 = (
    avgStreams
     .map(lambda x: (x[1], x[0]))
     # avgStreams, (year, artistId)
     .join(maxAvgStreams.map(lambda x: (x[1], None)))
     # avgStreams, ((year, artistId), None)
     .map(lambda x: (x[1][0][1], (x[1][0][0], x[0])))
     # artistId, (year, maxAvgStreams)
     .join(artistsRdd)
     # artistId, ((year, maxAvgStreams), artistName)
     .map(lambda x: (x[1][0][0], x[1][1], x[1][0][1]))
     # (year, artistName, maxAvgStreams)
     .sortBy(lambda x: (x[0], x[1]))
     # (year, artistName, maxAvgStreams)
)

times['query5']['rdd'] = timedRdd(rdd5, 'output/query5_rdd')

## Query 6

The given examples seem to be wrong.

```
viral50,2017,21 Savage,13
viral50,2017,Post Malone,13
```
For example the song `rockstar` (`songId = 178991`) by `Post Malone` and `21 Savage` does not have `13` consecutive days but `11`.

In [ ]:
# Proof
spark.sql('''
    SELECT *
    FROM charts
    JOIN chart_artist_mapping
    ON charts.songId = chart_artist_mapping.songId
    JOIN artists
    ON chart_artist_mapping.artistId = artists.artistId
    WHERE countryId = 23
        AND position = 1
        AND artists.artistName = '21 Savage'
        AND year(date) = 2017
        AND chartName = 'viral50'
    ORDER BY date
''').show(100)

In [ ]:
query6 = '''
    SELECT chartName, yr, artistName, maxStreak
    FROM (
        SELECT chartName, yr, songId, count(*) streak,
            max(count(*)) OVER (PARTITION BY chartName, yr) AS maxStreak
        FROM (
            SELECT chartName, year(date) yr, songId,
                date_sub(
                    date,
                    dense_rank() OVER (
                        PARTITION BY chartName, year(date), songId
                        ORDER BY date
                    )
                ) streakGrp
            FROM charts
            WHERE countryId = 23 AND position = 1
        )
        GROUP BY chartName, yr, songId, streakGrp
    ) a
    JOIN chart_artist_mapping
    ON a.songId = chart_artist_mapping.songId
    JOIN artists
    ON chart_artist_mapping.artistId = artists.artistId
    WHERE streak = maxStreak
    ORDER BY chartName, yr
'''

In [ ]:
useParquet()
times['query6']['parquet'] = timedSql(query6, 'output/query6_parquet')

In [ ]:
useCsv()
times['query6']['csv'] = timedSql(query6, 'output/query6_csv')

In [ ]:
def bestStreak(seq):
    seq = sorted(seq)
    res = 0
    streak = 1
    for prev, nxt in zip(seq, seq[1:]):
        if (nxt - prev).days == 1:
            streak += 1
        else:
            res = max(res, streak)
            streak = 1
    res = max(res, streak)
    return res


streaks = (
    chartsRdd
     .filter(lambda x: x[countryId] == '23' and x[position] == '1')
     .map(lambda x: ((x[chartName], parse_date(x[date]).year, x[songId]), (parse_date(x[date]),) ))
     # (chartName, year, songId), (date,)
     .reduceByKey(operator.add)
     # (chartName, year, songId), (date1, date2, ...)
     .mapValues(bestStreak)
     # (chartName, year, songId), streak
)

maxStreaks = (
    streaks
     .map(lambda x: (x[0][:2], x[1]))
     # (chartName, year), streak
     .reduceByKey(max)
     # (chartName, year), maxStreak
)

rdd6 = (
    streaks
     # (chartName, year, songId), maxStreak
     .map(lambda x: ((x[0][0], x[0][1], x[1]), x[0][2]))
     # (chartName, year, maxStreak), songId
     .join(maxStreaks.map(lambda x: ((*x[0], x[1]), None)))
     # (chartName, year, maxStreak), (songId, None)
     .map(lambda x: (x[1][0], x[0]))
     # songId, (chartName, year, maxStreak)
     .join(chart_artist_mappingRdd)
     # songID, ((chartName, year, maxStreak), artistId)
     .map(lambda x: (x[1][1], x[1][0]))
     # artistId, (chartName, year, maxStreak)
     .join(artistsRdd)
     # artistId, ((chartName, year, maxStreak), artistName)
     .map(lambda x: (x[1][0][0], x[1][0][1], x[1][1], x[1][0][2]))
     # chartName, year, artistName, maxStreak
     .sortBy(lambda x: x[:2])
     # chartName, year, artistName, maxStreak
)

times['query6']['rdd'] = timedRdd(rdd5, 'output/query6_rdd')

In [ ]:
df = pd.DataFrame.from_dict(times, orient='index')
df

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
df.plot.barh(ax=ax)
ax.set_xlabel('time (s)')
ax.set_title('Times')
ax.invert_yaxis()
plt.savefig('times.png')

In [ ]:
spark.stop()